## Let's start by setting up the environment.

In [ ]:
import os
import sagemaker

sagemaker_session = sagemaker.Session()

# Replace with a role (either name or full arn) that gives SageMaker access to S3 and cloudwatch
role='SageMakerRole'

## Downloading test and training data

In [ ]:
import utils

utils.cifar10_download()

## Uploading the data

In [ ]:
inputs = sagemaker_session.upload_data(path='/tmp/cifar10_data', key_prefix='data/cifar10')

# Complete source code

In [ ]:
!tree

# Running TensorFlow training on SageMaker

In [ ]:
from sagemaker.tensorflow import TensorFlow


sorce_dir = os.path.join(os.getcwd(), 'source_dir')
estimator = TensorFlow(entry_point='resnet_cifar_10.py',
                       source_dir=sorce_dir,
                       role=role,
                       hyperparameters={'training_steps': 1000, 'evaluation_steps': 100},
                       train_instance_count=2, train_instance_type='ml.p2.xlarge')

estimator.fit(inputs)

# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge')

# Deleting the endpoint

In [ ]:
estimator.delete_endpoint()